In [46]:
## Library
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime

In [47]:
#URL
URL = "https://www.blockchain.com/btc/unconfirmed-transactions"

def searchHTML(URL):
    #Request HTML & beautify it
    page = requests.get(URL)
    soup = bs(page.content)
    
    #Find right data
    results_numbers = soup.find_all("span", {'class' : 'sc-1ryi78w-0 cILyoi sc-16b9dsl-1 ZwupP u3ufsr-0 eQTRKC'})
    results_hash = soup.find_all("a", {'class' : 'sc-1r996ns-0 fLwyDF sc-1tbyx6t-1 kCGMTY iklhnl-0 eEewhk d53qjk-0 ctEFcK'})
    
    return results_numbers, results_hash

def makeFrame(hash, time, BTC, USD):
    dataF = {'Hash' : hash, 'Time' : time, 'BTC-value' : BTC, 'USD-value' : USD}
    frame = pd.DataFrame(data=dataF)
    
    return frame

def makeArrays(numbers, hash):
    only_time = [numbers[i] for i in range(0, len(numbers), 3)]
    only_BTC = [numbers[i] for i in range(1, len(numbers), 3)]
    only_USD = [numbers[i] for i in range(2, len(numbers), 3)]
    only_hash = [hash[i] for i in range(0, len(hash), 1)]
    
    return only_hash, only_time, only_BTC, only_USD

23:24

                                                 Hash     Time  \
0   [07c015180a1512685432b3f38ae8fe7cd5ab10b5a0c49...  [22:24]   
1   [946d22fb29f72c8e66aab5dc1a6f1fad088dadc0c6c90...  [22:24]   
2   [604b0d9efc3048ec1ffb4e28f3ddfb9d2a1b9cba64fed...  [22:24]   
3   [1fc7139725e1201f9c6e0a30bde2522f95af9096affe2...  [22:24]   
4   [b2d49e6bdeb3289661c5388f46cc67f85197148972c0d...  [22:24]   
5   [dda8f4ef96a4348b62a169487b3dfad3b9e8edaa29c57...  [22:24]   
6   [fab7f46dc7ba6c95143f34501a899cda01b981b409f8b...  [22:24]   
7   [549430ba45dc2f6f0dbae6ee90af44bfda5e1406f312b...  [22:24]   
8   [811f2617eb91ef6c39d685ad43fcd3cd30d85ea19614f...  [22:24]   
9   [54b163dd6a3db109ccc36de98b5de3425d331c3ac058d...  [22:24]   
10  [2cfabd78b2d27321e8af1a189e5d89c2c271ba52115a6...  [22:24]   
11  [f57d26d3631b77bc3c4d1c3ee3f182e9f646f6418cc2b...  [22:24]   
12  [d283a4df1caa618f42928ff34c15173208c07cffb2ba2...  [22:24]   
13  [fae85319fddf948cdc3c464548146c06acdb0235a2e0e...  [22:24]   
14 

In [ ]:
def fetchCurrent(URL):
    #HTML-doorzoeken en alles ophalen
    numbers, hash = searchHTML(URL)

    #Make arrays of only required values
    only_hash, only_time, only_BTC, only_USD = makeArrays(numbers, hash)

    #Make dataframe
    frame = makeFrame(only_hash, only_time, only_BTC, only_USD)

    #Tells current time
    print(f"{datetime.now().hour}:{datetime.now().minute}")

    #Print Dataframe
    print()
    print(frame)

fetchCurrent(URL)